In [ ]:
!pip install bitsandbytes -q
!pip install accelerate -q
!pip install transformers -q

In [ ]:
# =======================================================================================
#               INFERENCE SCRIPT FOR ERNIE-4.5-21B-A3B-Thinking (QUANTIZED)
#                          Adapted for Baidu's Model
# =======================================================================================

# @title Step 1: Install Dependencies
# Installs required libraries for 4-bit quantization and inference.
print(" Phase 1: Installing dependencies... Please wait. ⏳")
!pip install transformers bitsandbytes accelerate -q
print("Installation complete.\n")


# @title Step 2: Load Model and Tokenizer
# Loads the ERNIE model quantized in 4-bit.

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import time

print(" Phase 2: Loading ERNIE-4.5-21B-A3B-Thinking... Please wait.")

# Model Configuration
model_name = "baidu/ERNIE-4.5-21B-A3B-Thinking"
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load Quantized Model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

print("\nModel and Tokenizer loaded successfully.\n")


# @title Step 3: Prepare Prompt
# Setting up the initial text using the ERNIE format.

print(" Phase 3: Preparing the prompt...")

# Chat template for ERNIE
# Note: System prompt changed to a standard professional instruction.
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Explain the Unsloth library to me as if I were 5 years old."}
]

# Apply chat template
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

eos_token_id = tokenizer.eos_token_id

print("Prompt formatted and ready.\n")


# @title Step 4: Manual Inference with Streaming
# Generates text token by token and streams the output.

print("="*60)
print("Generating response...\n")

# -- Start Measurement --
start_time = time.time()

generated_tokens = 0
max_new_tokens = 256

# Tokenize initial prompt
inputs = tokenizer([formatted_prompt], return_tensors="pt").to(model.device)
input_ids_tensor = inputs.input_ids

# Store initial prompt length
prompt_token_length = input_ids_tensor.shape[1]

# Variable to store already printed text to avoid repetition
already_decoded_text = ""

# Use 'torch.no_grad()' to save memory and speed up inference
with torch.no_grad():
    for _ in range(max_new_tokens):
        # 1. Pass tokens to the model
        outputs = model(input_ids=input_ids_tensor)

        # 2. Get the most probable next token
        next_token_logits = outputs.logits[:, -1, :]
        next_token_id = torch.argmax(next_token_logits, dim=-1).unsqueeze(-1)

        # 3. Append new token to sequence
        input_ids_tensor = torch.cat([input_ids_tensor, next_token_id], dim=-1)

        # 4. Increment token count
        generated_tokens += 1

        # 5. Decode generated part so far
        tokens_generated_only = input_ids_tensor[0][prompt_token_length:]
        full_decoded_text = tokenizer.decode(tokens_generated_only, skip_special_tokens=True)

        # Print only the NEW part of the text
        new_text_chunk = full_decoded_text[len(already_decoded_text):]
        print(new_text_chunk, end='', flush=True)

        # Update printed text reference
        already_decoded_text = full_decoded_text

        # 6. Stop if model generates EOS token
        if next_token_id.item() == eos_token_id:
            break

# -- End Measurement --
end_time = time.time()
print("\n\n" + "="*60)
print("Generation complete.")

# --- Calculate and display results ---
total_time = end_time - start_time
tokens_per_second = (generated_tokens / total_time) if total_time > 0 else 0

print(f"\n **Performance Metrics** \n")
print(f" Tokens generated: {generated_tokens}")
print(f" Total generation time: {total_time:.2f} seconds")
print(f" **Speed: {tokens_per_second:.2f} tokens per second**")